In [1]:
data_dir = "data/raw_training/training_data"

In [2]:
import helpers.input_processor as ip 

data_dir = "data/raw_training/training_data"
cache_dir = 'data'

df = ip.loadTrainingData(data_dir, cache_dir, encode_data=False)

loading data from save file:  data/ingested_data.json


In [3]:
import helpers.input_processor as ip
import polars as pl
import numpy as np

#group together subjects for which the below features are identical
features = ['recording_location', 'age', 'sex', 'pregnancy_status', 'outcome']

filteredData = (
    df.groupby(['murmur_in_recording', *features]).agg([
        pl.count(),
        pl.col('spectrogram').list()
    ])
    .groupby(features).agg([
        pl.col(['murmur_in_recording', 'count', 'spectrogram']).list()
    ])
    .filter(pl.col('count').arr.lengths()==2)
    .sort(pl.col('count').arr.min(), reverse=True)
    .with_row_count('r_index')
)

filteredData.head(10)

r_index,recording_location,age,sex,pregnancy_status,outcome,murmur_in_recording,count,spectrogram
u32,str,str,str,str,str,list [str],list [u32],list [list [str]]
0,"""TV""","""Child""","""Female""","""False""","""Abnormal""","[""Present"", ""Absent""]","[47, 92]","[[""data/spectrograms/29045_TV.npy"", ""data/spectrograms/33151_TV.npy"", ... ""data/spectrograms/9979_TV.npy""], [""data/spectrograms/23625_TV.npy"", ""data/spectrograms/24160_TV.npy"", ... ""data/spectrograms/85319_TV.npy""]]"
1,"""MV""","""Child""","""Female""","""False""","""Abnormal""","[""Present"", ""Absent""]","[46, 109]","[[""data/spectrograms/29045_MV.npy"", ""data/spectrograms/33151_MV.npy"", ... ""data/spectrograms/9979_MV.npy""], [""data/spectrograms/23625_MV.npy"", ""data/spectrograms/24160_MV.npy"", ... ""data/spectrograms/85319_MV.npy""]]"
2,"""TV""","""Child""","""Male""","""False""","""Abnormal""","[""Present"", ""Absent""]","[43, 97]","[[""data/spectrograms/13918_TV.npy"", ""data/spectrograms/14241_TV.npy"", ... ""data/spectrograms/85172_TV.npy""], [""data/spectrograms/14998_TV.npy"", ""data/spectrograms/40798_TV.npy"", ... ""data/spectrograms/9983_TV.npy""]]"
3,"""MV""","""Child""","""Male""","""False""","""Abnormal""","[""Present"", ""Absent""]","[42, 108]","[[""data/spectrograms/14241_MV.npy"", ""data/spectrograms/46065_MV.npy"", ... ""data/spectrograms/85172_MV.npy""], [""data/spectrograms/13918_MV.npy"", ""data/spectrograms/14998_MV.npy"", ... ""data/spectrograms/9983_MV.npy""]]"
4,"""PV""","""Child""","""Female""","""False""","""Abnormal""","[""Present"", ""Absent""]","[42, 97]","[[""data/spectrograms/29045_PV.npy"", ""data/spectrograms/49627_PV.npy"", ... ""data/spectrograms/9979_PV.npy""], [""data/spectrograms/23625_PV.npy"", ""data/spectrograms/24160_PV.npy"", ... ""data/spectrograms/85343_PV.npy""]]"
5,"""PV""","""Child""","""Male""","""False""","""Abnormal""","[""Present"", ""Absent""]","[40, 102]","[[""data/spectrograms/14241_PV.npy"", ""data/spectrograms/46065_PV.npy"", ... ""data/spectrograms/85172_PV.npy""], [""data/spectrograms/13918_PV.npy"", ""data/spectrograms/14998_PV.npy"", ... ""data/spectrograms/9983_PV.npy""]]"
6,"""AV""","""Child""","""Female""","""False""","""Abnormal""","[""Absent"", ""Present""]","[109, 39]","[[""data/spectrograms/23625_AV.npy"", ""data/spectrograms/24160_AV.npy"", ... ""data/spectrograms/85343_AV.npy""], [""data/spectrograms/29045_AV.npy"", ""data/spectrograms/49595_AV.npy"", ... ""data/spectrograms/9979_AV.npy""]]"
7,"""AV""","""Child""","""Male""","""False""","""Abnormal""","[""Present"", ""Absent""]","[30, 107]","[[""data/spectrograms/14241_AV.npy"", ""data/spectrograms/46065_AV.npy"", ... ""data/spectrograms/85172_AV.npy""], [""data/spectrograms/13918_AV.npy"", ""data/spectrograms/14998_AV.npy"", ... ""data/spectrograms/9983_AV.npy""]]"
8,"""MV""","""Infant""","""Female""","""False""","""Abnormal""","[""Present"", ""Absent""]","[13, 26]","[[""data/spectrograms/49824_MV.npy"", ""data/spectrograms/49900_MV.npy"", ... ""data/spectrograms/85276_MV.npy""], [""data/spectrograms/49607_MV.npy"", ""data/spectrograms/49622_MV.npy"", ... ""data/spectrograms/85038_MV.npy""]]"


In [4]:
#choose which group to analyze (by index)
selectedRow = filteredData.slice(0,1)

#get relevant spectrograms (including presence of a murmur) for the selected set of data
comparableData = pl.DataFrame(
    data = selectedRow.get_column('spectrogram').to_list(),
    columns = selectedRow.get_column('murmur_in_recording')[0].to_list()
)
#choose equal amounts of spectrograms for both 'Absent' and 'Present' cases
x = min(comparableData.select(pl.all().arr.lengths()).row(0))
comparableData = comparableData.with_column(
    pl.all()
    .arr.eval(pl.element().slice(0,x))
).explode(pl.all())

comparableData.head()

Present,Absent
str,str
"""data/spectrograms/29045_TV.npy""","""data/spectrograms/23625_TV.npy"""
"""data/spectrograms/33151_TV.npy""","""data/spectrograms/24160_TV.npy"""
"""data/spectrograms/49627_TV.npy""","""data/spectrograms/2530_TV.npy"""
"""data/spectrograms/49754_TV.npy""","""data/spectrograms/31737_TV.npy"""
"""data/spectrograms/49821_TV.npy""","""data/spectrograms/39043_TV.npy"""


In [5]:
import os
import polars as pl
import numpy as np
import helpers.audio_tools as adt
import matplotlib.pyplot as plt
import tqdm

dir = 'comparing_spectrograms'

#save to a file the list of features that all data being analyzed has in common

os.makedirs(dir, exist_ok=True)
with open(dir + '/shared_features.txt', 'w') as f:
    printout = selectedRow.select(features).to_dict(as_series=False)
    lines = [
        'features shared by all spectrograms stored in murmur_present and murmur_absent directories:',
        '{',
        *['    %s: %s' % (key, value) for key, value in printout.items()],
        '}'
    ]
    f.write('\n'.join(lines))

#save MURMUR PRESENT spectro images to files

dir_p = dir + '/murmur_present'
spectros_p = comparableData.get_column('Present').to_list()

os.makedirs(dir_p, exist_ok=True)

progress = tqdm.tqdm(total=comparableData.height, desc='saving spectro images to files - PRESENT')
for x in spectros_p:
    spectro = np.load(x)
    name = x.split('/')[-1].replace('.npy', '')
    adt.printSpectro(spectro, title=name+' - Murmur PRESENT in Recording')
    plt.savefig(dir_p + '/' + name + '.png', facecolor='white', bbox_inches='tight', pad_inches=0)
    plt.close()
    progress.update(1)
progress.close()

#save MURMUR ABSENT spectro images to files

dir_a = dir + '/murmur_absent'
spectros_a = comparableData.get_column('Absent').to_list()

os.makedirs(dir_a, exist_ok=True)

progress = tqdm.tqdm(total=comparableData.height, desc='saving spectro images to files - ABSENT')
for x in spectros_a:
    spectro = np.load(x)
    name = x.split('/')[-1].replace('.npy', '')
    adt.printSpectro(spectro, title=name+' - Murmur ABSENT in Recording')
    plt.savefig(dir_a + '/' + name + '.png', facecolor='white', bbox_inches='tight', pad_inches=0)
    plt.close()
    progress.update(1)
progress.close()

saving spectro images to files - ABSENT: 100%|██████████| 47/47 [00:36<00:00,  1.29it/s]


In [18]:
df.height

3163